In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from distances import hello

hello()

'alv'

In [ ]:
# script for calculate interatomic distances intraprotein (same chain)
# between heavy atoms of Kinase domain (KD) and Cis-regulatory elements (CRE)

In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
# Datasets
msa_regions_pdbs = pd.read_csv('../datasets/msa_regions_pdbs.tsv', sep= '\t')
mapping_coordinates_uniprot_pdb = pd.read_csv('../datasets/mapping_coordinates_uniprot_pdb.tsv', sep = '\t')

In [ ]:
# # Functions for uncompress pdb files from batch download
# def is_gz_file(filepath):
#     '''
#     detects a gzip compressed file by testing if the first two bytes are 1f 8b.
#     '''
#     with open(filepath, 'rb') as test_f:
#         return test_f.read(2) == b'\x1f\x8b'

# def uncompress_pdb(filepath, output):
#     '''
#     Uncompress gz pdb files
#     '''
#     import gzip
#     import shutil
#     with gzip.open(filepath, 'rb') as f_in:
#         with open(output, 'wb') as f_out:
#             shutil.copyfileobj(f_in, f_out)
    
#     # import gzip
#     # with gzip.open(filepath, 'rb') as f:
#     #     return f.read()

In [ ]:
# # Uncompress pdb files
# path= '../datasets/pdb_files/'
# if not os.path.isdir(path + "uncompressed"):
#     os.mkdir(path + "uncompressed")
# for filename in os.listdir(path):
#     f = os.path.join(path, filename) # is the complete path to the pdb file
#     #print(f)
#     filename = filename.split(".")[0] # the first element is the pdb accession
#     #print(filename)
#     if is_gz_file(f):
#         output = os.path.join(path + "uncompressed", filename + ".pdb")
#         try:
#             uncompress_pdb(filepath= f, output= output) # uncompressed file
#         except:
#             print(f"{filename} something else went wrong")
#     else:
#         print(f"Can not uncompress file {f}")

In [19]:
# Tomar todas las distancias de los atomos pesados, no filtrar por threshold
def distances_all_atoms(path, output, pdb_coord): # threshold = 6
    '''
    Given a pdb file, calculates the interatomic distances among heavy atoms
    Returns: a data frame with interatomic distances
    path: path containing PDB files
    output: output path
    pdb_coord: pandas DataFrame containing regions in PDB coordinates
    '''
    import os
    import distances
    from distances import all_atoms_selector
    from Bio.PDB.PDBParser import PDBParser
    import warnings

    warnings.simplefilter("ignore")
    # iterate over the pdb files
    for filename in os.listdir(path):
        f = os.path.join(path, filename) # is the complete path to the pdb file

        if os.path.isfile(f):
            #print(f'{f} is file')
            try:
                pdb = PDBParser().get_structure(str(filename), path + filename)
                filename = filename.rstrip(".pdb") # get pdb accesion
                #print(f"in pdb: {filename}")
                
                if filename in pdb_coord.pdb.unique().tolist():
                    output_file = output + filename + ".csv"
                    #print(f'output file: {output_file}')
                    
                    # iterate over each chain in the dataframe
                    chains_kd_cre = {}
                    for chain in pdb_coord[pdb_coord.pdb == filename].chain.drop_duplicates():
                        #print(f'in chain {chain}')
                        df_chains = pdb_coord[(pdb_coord.pdb == filename) & (pdb_coord.chain == chain)]
                        pdb_positions = set(eval(df_chains.mapping.values[0]).values()) # tomar el primero, ya que ese pdb puede estar en distintos MSAs
                        #print(pdb_positions)
                        
                        kd = set()
                        cre = set()

                        kd = pdb_positions.intersection(set([i for i in range(df_chains.pdb_start_kd.values[0], df_chains.pdb_end_kd.values[0] + 1)])) ### toamr del mapeo
                        cre = pdb_positions.intersection(set([i for i in range(df_chains.pdb_start_cre.values[0], df_chains.pdb_end_cre.values[0] + 1)]))
                        #print(f"set: {set([i for i in range(df_chains.pdb_start_kd.values[0], df_chains.pdb_end_kd.values[0] + 1)])}")
                        #print(f"kd:{kd}")
                        chains_kd_cre[chain] = {"kd": sorted(list(kd)), "cre": sorted(list(cre))}
                        #print(chains_kd_cre)

                    # check if output already exists
                    if not os.path.exists(output_file):
                        #print("inside path_output")
                        # calculate distance
                        # param pdb_source: a path to a pdb file, a Bio.PDB.Structure or a Bio.PDB.Model
                        dist = distances.calculate_distances(
                            pdb_source= f, # cambiar por archivo q no contenga los hetatoms
                            chains_kd_cre= chains_kd_cre,
                            atom_selector= all_atoms_selector,
                            include_extra_info= True
                        )
                        print(f"distances for {filename} calculated")
                        # df with distances filtered by threshold
                        df = pd.DataFrame(dist, columns = ['chain_a', 'pos_a', 'aa_a', 'atom_a', 'chain_b', 'pos_b', 'aa_b', 'atom_b', 'dist'])
                        df = df[(df.pos_a > 0) & (df.pos_b > 0)]
                        #df = df[df.dist <= threshold]
                        
                        #df["uniprot"] = uniprot
                        df["pdb"] = filename
                        print(f"writing distances as {filename}.csv")
                        df.to_csv(output_file, index= False)
            except BaseException as e:
                #print(f'An error ocurred for {filename}')
                print(str(e))

In [3]:
# Generate a dataframe with the positions of CRE and KD in PDB
mapping_df = mapping_coordinates_uniprot_pdb[['uniprot', 'pdb', 'chain', 'mapping']]
regions_df = msa_regions_pdbs[['msa', 'uniprot', 'pdb','term_id_cre', 'start_cre', 'end_cre', 'term_id_kd', 'start_kd', 'end_kd']]

# Merge both tables based on uniprot-pdb
merged_df = regions_df.merge(mapping_df, on=["uniprot", "pdb"])
# Create columns for coordinates in pdb
merged_df["pdb_start_cre"] = 0
merged_df["pdb_end_cre"] = 0
merged_df["pdb_start_kd"] = 0
merged_df["pdb_end_kd"] = 0

for index, row in merged_df.iterrows():
    start_cre = row["start_cre"]
    end_cre = row["end_cre"]
    start_kd = row["start_kd"]
    end_kd = row["end_kd"]
    # eval() function is used to convert the string representation into a dictionary object, which is stored in the mapping variable.
    mapping = eval(row["mapping"])
    if index == 27:
        print(index)
        print(mapping)

    min_mapping = min(mapping.keys())
    max_mapping = max(mapping.keys())

    # get the coordinates in pdb using the mapping
    # los start buscan forward if NaN a new start, ends look backwards for a new end if NaN
    while start_cre <= max_mapping:
        pdb_start_cre = mapping.get(start_cre, np.nan)
        if np.isnan(pdb_start_cre):
            start_cre += 1
        else:
            break
    
    while end_cre >= min_mapping: 
        pdb_end_cre = mapping.get(end_cre, np.nan)
        if np.isnan(pdb_end_cre):
            end_cre -= 1
        else:
            break

    while start_kd <= max_mapping:
        pdb_start_kd = mapping.get(start_kd, np.nan)
        if np.isnan(pdb_start_kd):
            start_kd += 1
        else:
            break

    while end_kd >= min_mapping: 
        pdb_end_kd = mapping.get(end_kd, np.nan)
        if np.isnan(pdb_end_kd):
            end_kd -= 1
        else:
            break

    # # Add a check to make sure that pdb_start_* is less than or equal to pdb_end_*
    # if pdb_start_cre > pdb_end_cre:
    #     pdb_start_cre, pdb_end_cre = pdb_end_cre, pdb_start_cre
    # if pdb_start_kd > pdb_end_kd:
    #     pdb_start_kd, pdb_end_kd = pdb_end_kd, pdb_start_kd

    # the coordinates in pdb are written back to their respective columns
    merged_df.at[index, "pdb_start_cre"] = pdb_start_cre
    merged_df.at[index, "pdb_end_cre"] = pdb_end_cre
    merged_df.at[index, "pdb_start_kd"] = pdb_start_kd
    merged_df.at[index, "pdb_end_kd"] = pdb_end_kd


27
{300: 300, 301: 301, 302: 302, 303: 303, 304: 304, 305: 305, 306: 306, 307: 307, 308: 308, 309: 309, 310: 310, 311: 311, 312: 312, 313: 313, 314: 314, 315: 315, 316: 316, 317: 317, 318: 318, 319: 319, 320: 320, 321: 321, 322: 322, 323: 323, 324: 324, 325: 325, 326: 326, 327: 327, 328: 328, 329: 329, 330: 330, 331: 331, 332: 332, 333: 333, 334: 334, 335: 335, 336: 336, 337: 337, 338: 338, 339: 339, 340: 340, 341: 341, 342: 342, 343: 343, 344: 344, 345: 345, 346: 346, 347: 347, 348: 348, 349: 349, 350: 350, 351: 351, 352: 352, 353: 353, 354: 354, 355: 355, 356: 356, 357: 357, 358: 358, 359: 359, 360: 360, 361: 361, 362: 362, 363: 363, 364: 364, 365: 365, 366: 366, 367: 367, 368: 368, 369: 369, 370: 370, 371: 371, 372: 372, 373: 373, 374: 374, 375: 375, 376: 376, 377: 377, 378: 378, 379: 379, 380: 380, 381: 381, 382: 382, 383: 383, 384: 384, 385: 385, 386: 386, 387: 387, 388: 388, 389: 389, 390: 390, 391: 391, 392: 392, 393: 393, 394: 394, 395: 395, 396: 396, 397: 397, 398: 398, 399: 3

In [4]:
merged_df[['msa', 'uniprot', 'pdb', 'chain', 'term_id_cre', 'start_cre', 'end_cre',
       'term_id_kd', 'start_kd', 'end_kd',  'pdb_start_cre',
       'pdb_end_cre', 'pdb_start_kd', 'pdb_end_kd']].dropna() # Ok! no pierdo casos

,msa,uniprot,pdb,chain,term_id_cre,start_cre,end_cre,term_id_kd,start_kd,end_kd,pdb_start_cre,pdb_end_cre,pdb_start_kd,pdb_end_kd
0,A5K0N4_60,A5K0N4,5f0a,A,cre10,1,22,kd10,534,791,1,22,534,791
1,Q8I719_60,A5K0N4,5f0a,A,cre10,1,22,kd10,534,791,1,22,534,791
2,A5K0N4_60,A5K0N4,5dzc,A,cre10,1,22,kd10,534,791,1,22,534,791
3,Q8I719_60,A5K0N4,5dzc,A,cre10,1,22,kd10,534,791,1,22,534,791
4,A5K0N4_60,A5K0N4,5dyl,A,cre10,1,22,kd10,534,791,1,22,534,791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,Q9UIK4_60,Q9UIK4,1z9x,A,cre273,292,301,kd273,23,285,282,291,13,275
1074,Q8VDF3_60,Q9UIK4,6paw,A,cre273,292,301,kd273,23,285,282,291,13,275
1075,Q9UIK4_60,Q9UIK4,6paw,A,cre273,292,301,kd273,23,285,282,291,13,275
1076,Q91VB2_60,Q96NX5,2jam,A,cre243,277,317,kd243,23,277,278,303,23,278


In [5]:
merged_df.pdb.nunique()

828

---  
### checks

In [6]:
mapping_df[mapping_df.pdb == '2hz0'].mapping.values

array(['{233: 1, 234: 2, 235: 3, 236: 4, 237: 5, 238: 6, 239: 7, 240: 8, 241: 9, 242: 10, 243: 11, 244: 12, 245: 13, 246: 14, 247: 15, 248: 16, 249: 17, 250: 18, 251: 19, 252: 20, 253: 21, 254: 22, 255: 23, 256: 24, 257: 25, 258: 26, 259: 27, 260: 28, 261: 29, 262: 30, 263: 31, 264: 32, 265: 33, 266: 34, 267: 35, 268: 36, 269: 37, 270: 38, 271: 39, 272: 40, 273: 41, 274: 42, 275: 43, 276: 44, 277: 45, 278: 46, 279: 47, 280: 48, 281: 49, 282: 50, 283: 51, 284: 52, 285: 53, 286: 54, 287: 55, 288: 56, 289: 57, 290: 58, 291: 59, 292: 60, 293: 61, 294: 62, 295: 63, 296: 64, 297: 65, 298: 66, 299: 67, 300: 68, 301: 69, 302: 70, 303: 71, 304: 72, 305: 73, 306: 74, 307: 75, 308: 76, 309: 77, 310: 78, 311: 79, 312: 80, 313: 81, 314: 82, 315: 83, 316: 84, 317: 85, 318: 86, 319: 87, 320: 88, 321: 89, 322: 90, 323: 91, 324: 92, 325: 93, 326: 94, 327: 95, 328: 96, 329: 97, 330: 98, 331: 99, 332: 100, 333: 101, 334: 102, 335: 103, 336: 104, 337: 105, 338: 106, 339: 107, 340: 108, 341: 109, 342: 110,

In [37]:
mapping_coordinates_uniprot_pdb[mapping_coordinates_uniprot_pdb.pdb == '2hz0'].alignment.values

array(["Alignment(seqA='MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKQGVRGAVSTLLQAPELPTKTRTSRRAAEHRDTTDVPEMPHSKGQGESDPLDHEPAVSPLLPRKERGPPEGGLNEDERLLPKDKKTNLFSALIKKKKKTAPTPPKRSSSFREMDGQPERRGAGEEEGRDISNGALAFTPLDTADPAKSPKPSNGAGVPNGALRESGGSGFRSPHLWKKSSTLTSSRLATGEEEGGGSSSKRFLRSCSASCVPHGAKDTEWRSVTLPRDLQSTGRQFDSSTFGGHKSEKPALPRKRAGENRSDQVTRGTVTPPPRLVKKNEEAADEVFKDIMESSPGSSPPNLTPKPLRRQVTVAPASGLPHKEEAGKGSALGTPAAAEPVTPTSKAGSGAPGGTSKGPAEESRVRRHKHSSESPGRDKGKLSRLKPAPPPPPAASAGKAGGKPSQSPSQEAAGEAVLGAKTKATSLVDAVNSDAAKPSQPGEGLKKPVLPATPKPQSAKPSGTPI

In [36]:
pdb_sequences = pd.read_csv('../datasets/pdb_sequences.tsv', sep = '\t')

In [40]:
pdb_sequences[pdb_sequences.pdb == '2hz0'].sequence.values[0]

'DKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMF'

In [ ]:
mapping_coordinates_uniprot_pdb[mapping_coordinates_uniprot_pdb.pdb == '2hz0'].pdb_seq.values[0]

In [41]:
from Bio.Seq import Seq
uniprot_p00519 = Seq("MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKQGVRGAVSTLLQAPELPTKTRTSRRAAEHRDTTDVPEMPHSKGQGESDPLDHEPAVSPLLPRKERGPPEGGLNEDERLLPKDKKTNLFSALIKKKKKTAPTPPKRSSSFREMDGQPERRGAGEEEGRDISNGALAFTPLDTADPAKSPKPSNGAGVPNGALRESGGSGFRSPHLWKKSSTLTSSRLATGEEEGGGSSSKRFLRSCSASCVPHGAKDTEWRSVTLPRDLQSTGRQFDSSTFGGHKSEKPALPRKRAGENRSDQVTRGTVTPPPRLVKKNEEAADEVFKDIMESSPGSSPPNLTPKPLRRQVTVAPASGLPHKEEAGKGSALGTPAAAEPVTPTSKAGSGAPGGTSKGPAEESRVRRHKHSSESPGRDKGKLSRLKPAPPPPPAASAGKAGGKPSQSPSQEAAGEAVLGAKTKATSLVDAVNSDAAKPSQPGEGLKKPVLPATPKPQSAKPSGTPISPAPVPSTLPSASSALAGDQPSSTAFIPLISTRVSLRKTRQPPERIASGAITKGVVLDSTEALCLAISRNSEQMASHSAVLEAGKNLYTFCVSYVDSIQQMRNKFAFREAINKLENNLRELQICPATAGSGPAATQDFSKLLSSVKEISDIVQR")
pdb_2hz0 = Seq(pdb_sequences[pdb_sequences.pdb == '2hz0'].sequence.values[0])

In [42]:
from Bio import pairwise2
alignment = pairwise2.align.globalms(uniprot_p00519, pdb_2hz0, 2, -1, -.5, -.1)

In [43]:
# check the alignment
print(pairwise2.format_alignment(*alignment[0]))

MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAARWNSKENLLAGPSENDPNLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVNSLEKHSWYHGPVSRNAAEYLLSSGINGSFLVRESESSPGQRSISLRYEGRVYHYRINTASDGKLYVSSESRFNTLAELVHHHSTVADGLITTLHYPAPKRNKPTVYGVSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMFQESSISDEVEKELGKQGVRGAVSTLLQAPELPTKTRTSRRAAEHRDTTDVPEMPHSKGQGESDPLDHEPAVSPLLPRKERGPPEGGLNEDERLLPKDKKTNLFSALIKKKKKTAPTPPKRSSSFREMDGQPERRGAGEEEGRDISNGALAFTPLDTADPAKSPKPSNGAGVPNGALRESGGSGFRSPHLWKKSSTLTSSRLATGEEEGGGSSSKRFLRSCSASCVPHGAKDTEWRSVTLPRDLQSTGRQFDSSTFGGHKSEKPALPRKRAGENRSDQVTRGTVTPPPRLVKKNEEAADEVFKDIMESSPGSSPPNLTPKPLRRQVTVAPASGLPHKEEAGKGSALGTPAAAEPVTPTSKAGSGAPGGTSKGPAEESRVRRHKHSSESPGRDKGKLSRLKPAPPPPPAASAGKAGGKPSQSPSQEAAGEAVLGAKTKATSLVDAVNSDAAKPSQPGEGLKKPVLPATPKPQSAKPSGTPISPAPVPSTLPSASSALAGDQPSST

In [11]:
pdb_sequences = pd.read_csv('../datasets/pdb_sequences.tsv', sep= '\t')
pdb_sequences[pdb_sequences.pdb == '2hz0'].sequence.values

array(['DKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTLKEDTMEVEEFLKEAAVMKEIKHPNLVQLLGVCTREPPFYIITEFMTYGNLLDYLRECNRQEVNAVVLLYMATQISSAMEYLEKKNFIHRDLAARNCLVGENHLVKVADFGLSRLMTGDTYTAHAGAKFPIKWTAPESLAYNKFSIKSDVWAFGVLLWEIATYGMSPYPGIDLSQVYELLEKDYRMERPEGCPEKVYELMRACWQWNPSDRPSFAEIHQAFETMF'],
      dtype=object)

In [12]:
pdb_sequences[pdb_sequences.pdb == '2hz0'].pos_aa.values

array(["[(233, 'D'), (234, 'K'), (235, 'W'), (236, 'E'), (237, 'M'), (238, 'E'), (239, 'R'), (240, 'T'), (241, 'D'), (242, 'I'), (243, 'T'), (244, 'M'), (245, 'K'), (246, 'H'), (247, 'K'), (248, 'L'), (249, 'G'), (250, 'G'), (251, 'G'), (252, 'Q'), (253, 'Y'), (254, 'G'), (255, 'E'), (256, 'V'), (257, 'Y'), (258, 'E'), (259, 'G'), (260, 'V'), (261, 'W'), (262, 'K'), (263, 'K'), (264, 'Y'), (265, 'S'), (266, 'L'), (267, 'T'), (268, 'V'), (269, 'A'), (270, 'V'), (271, 'K'), (272, 'T'), (273, 'L'), (274, 'K'), (275, 'E'), (276, 'D'), (277, 'T'), (278, 'M'), (279, 'E'), (280, 'V'), (281, 'E'), (282, 'E'), (283, 'F'), (284, 'L'), (285, 'K'), (286, 'E'), (287, 'A'), (288, 'A'), (289, 'V'), (290, 'M'), (291, 'K'), (292, 'E'), (293, 'I'), (294, 'K'), (295, 'H'), (296, 'P'), (297, 'N'), (298, 'L'), (299, 'V'), (300, 'Q'), (301, 'L'), (302, 'L'), (303, 'G'), (304, 'V'), (305, 'C'), (306, 'T'), (307, 'R'), (308, 'E'), (309, 'P'), (310, 'P'), (311, 'F'), (312, 'Y'), (313, 'I'), (314, 'I'), (315, '

---

### more checks

In [49]:
mapping_coordinates_uniprot_pdb[mapping_coordinates_uniprot_pdb.pdb == '6ty3'].mapping[747]

'{35: 1, 36: 2, 37: 3, 38: 4, 39: 5, 40: 6, 41: 7, 42: 8, 43: 9, 44: 10, 45: 11, 46: 12, 47: 13, 48: 14, 49: 15, 50: 16, 51: 17, 52: 18, 53: 19, 54: 20, 55: 21, 56: 22, 57: 23, 58: 24, 59: 25, 60: 26, 61: 27, 62: 28, 63: 29, 64: 30, 65: 31, 66: 32, 67: 33, 68: 34, 69: 35, 70: 36, 71: 37, 72: 38, 73: 39, 74: 40, 75: 41, 76: 42, 77: 43, 78: 44, 79: 45, 80: 46, 81: 47, 82: 48, 83: 49, 84: 50, 85: 51, 86: 52, 87: 53, 88: 54, 89: 55, 90: 56, 91: 57, 92: 58, 93: 59, 94: 60, 95: 61, 96: 62, 97: 63, 98: 64, 99: 65, 100: 66, 101: 67, 102: 68, 103: 69, 104: 70, 105: 71, 106: 72, 107: 73, 108: 74, 109: 75, 110: 76, 111: 77, 112: 78, 113: 79, 114: 80, 115: 81, 116: 82, 117: 83, 118: 84, 119: 85, 120: 86, 121: 87, 122: 88, 123: 89, 124: 90, 125: 91, 126: 92, 127: 93, 128: 94, 129: 95, 130: 96, 131: 97, 132: 98, 133: 99, 134: 100, 135: 101, 136: 102, 137: 103, 138: 104, 139: 105, 140: 106, 141: 107, 142: 108, 143: 109, 144: 110, 145: 111, 146: 112, 147: 113, 148: 114, 149: 115, 150: 116, 151: 117, 1

In [55]:
mapping_coordinates_uniprot_pdb[mapping_coordinates_uniprot_pdb.pdb == '6ty3'].alignment[747]

"Alignment(seqA='MAAAYLDPNLNHTPSSSAKTHLGTGMERSPGAMERVLKVFHYFENSSEPTTWASIIRHGDATDVRGIIQKIVDCHKVKNVACYGLRLSHLQSEEVHWLHLDMGVSNVREKFELAHPPEEWKYELRIRYLPKGFLNQFTEDKPTLNFFYQQVKNDYMLEIADQVDQEIALKLGCLEIRRSYGEMRGNALEKKSNYEVLEKDVGLRRFFPKSLLDSVKAKTLRKLIQQTFRQFANLNREESILKFFEILSPVYRFDKECFKCALGSSWIISVELAIGPEEGISYLTDKGANPTHLADFNQVQTIQYSNSEDKDRKGMLQLKIAGAPEPLTVTAPSLTIAENMADLIDGYCRLVNGATQSFIIRPQKEGERALPSIPKLANNEKQGVRSHTVSVSETDDYAEIIDEEDTYTMPSTRDYEIQRERIELGRCIGEGQFGDVHQGIYMSPENPAMAVAIKTCKNCTSDSVREKFLQEALTMRQFDHPHIVKLIGVITENPVWIIMELCTLGELRSFLQVRKFSLDLASLILYAYQLSTALAYLESKRFVHRDIAARNVLVSATDCVKLGDFGLSRYMEDSTYYKASKGKLPIKWMAPESINFRRFTSASDVWMFGVCMWEILMHGVKPFQGVKNNDVIGRIENGERLPMPPNCPPTLYSLMTKCWAYDPSRRPRFTELKAQLSTILEEEKLQQEERMRMESRRQVTVSWDSGGSDEAPPKPSRPGYPSPRSSEGFYPSPQHMVQPNHYQVSGYSGSHGIPAMAGSIYPGQASLLDQTDSWNHRPQEVSAWQPNMEDSGTLDVRGMGQVLPTHLMEERLIRQQQEMEEDQRWLEKEERFLVMKPDVRLSRGSIEREDGGLQGPAGNQHIYQPVGKPDHAAPPKKPPRPGAPHLGSLASLNSPVDSYNEGVKIKPQEISPPPTANLDRSNDKVYENVTGLVKAVIEMSSKIQPAPPEEYVPMVKEVGLALRTLLATVDESLPVLPASTHRE

In [54]:
print(pairwise2.format_alignment(*alignment[0]))

TypeError: format_alignment() missing 4 required positional arguments: 'align2', 'score', 'begin', and 'end'

In [63]:
mapping = {35: 1, 36: 2, 37: 3, 38: 4, 39: 5, 40: 6, 41: 7, 42: 8, 43: 9, 44: 10, 45: 11, 46: 12, 47: 13, 48: 14, 49: 15, 50: 16, 51: 17, 52: 18, 53: 19, 54: 20, 55: 21, 56: 22, 57: 23, 58: 24, 59: 25, 60: 26, 61: 27, 62: 28, 63: 29, 64: 30, 65: 31, 66: 32, 67: 33, 68: 34, 69: 35, 70: 36, 71: 37, 72: 38, 73: 39, 74: 40, 75: 41, 76: 42, 77: 43, 78: 44, 79: 45, 80: 46, 81: 47, 82: 48, 83: 49, 84: 50, 85: 51, 86: 52, 87: 53, 88: 54, 89: 55, 90: 56, 91: 57, 92: 58, 93: 59, 94: 60, 95: 61, 96: 62, 97: 63, 98: 64, 99: 65, 100: 66, 101: 67, 102: 68, 103: 69, 104: 70, 105: 71, 106: 72, 107: 73, 108: 74, 109: 75, 110: 76, 111: 77, 112: 78, 113: 79, 114: 80, 115: 81, 116: 82, 117: 83, 118: 84, 119: 85, 120: 86, 121: 87, 122: 88, 123: 89, 124: 90, 125: 91, 126: 92, 127: 93, 128: 94, 129: 95, 130: 96, 131: 97, 132: 98, 133: 99, 134: 100, 135: 101, 136: 102, 137: 103, 138: 104, 139: 105, 140: 106, 141: 107, 142: 108, 143: 109, 144: 110, 145: 111, 146: 112, 147: 113, 148: 114, 149: 115, 150: 116, 151: 117, 152: 118, 153: 119, 154: 120, 155: 121, 156: 122, 157: 123, 158: 124, 159: 125, 160: 126, 161: 127, 162: 128, 163: 129, 164: 130, 165: 131, 166: 132, 167: 133, 168: 134, 169: 135, 170: 136, 171: 137, 172: 138, 173: 139, 174: 140, 175: 141, 176: 142, 177: 143, 178: 144, 179: 145, 180: 146, 181: 147, 182: 148, 183: 149, 184: 150, 185: 151, 186: 152, 187: 153, 188: 154, 189: 155, 190: 156, 191: 157, 192: 158, 193: 159, 194: 160, 195: 161, 196: 162, 197: 163, 198: 164, 199: 165, 200: 166, 201: 167, 202: 168, 203: 169, 204: 170, 205: 171, 206: 172, 207: 173, 208: 174, 209: 175, 210: 176, 211: 177, 212: 178, 213: 179, 214: 180, 215: 181, 216: 182, 217: 183, 218: 184, 219: 185, 220: 186, 221: 187, 222: 188, 223: 189, 224: 190, 225: 191, 226: 192, 227: 193, 228: 194, 229: 195, 230: 196, 231: 197, 232: 198, 233: 199, 234: 200, 235: 201, 236: 202, 237: 203, 238: 204, 239: 205, 240: 206, 241: 207, 242: 208, 243: 209, 244: 210, 245: 211, 246: 212, 247: 213, 248: 214, 249: 215, 250: 216, 251: 217, 252: 218, 253: 219, 254: 220, 255: 221, 256: 222, 257: 223, 258: 224, 259: 225, 260: 226, 261: 227, 262: 228, 263: 229, 264: 230, 265: 231, 266: 232, 267: 233, 268: 234, 269: 235, 270: 236, 271: 237, 272: 238, 273: 239, 274: 240, 275: 241, 276: 242, 277: 243, 278: 244, 279: 245, 280: 246, 281: 247, 282: 248, 283: 249, 284: 250, 285: 251, 286: 252, 287: 253, 288: 254, 289: 255, 290: 256, 291: 257, 292: 258, 293: 259, 294: 260, 295: 261, 296: 262, 297: 263, 298: 264, 299: 265, 300: 266, 301: 267, 302: 268, 303: 269, 304: 270, 305: 271, 306: 272, 307: 273, 308: 274, 309: 275, 310: 276, 311: 277, 312: 278, 313: 279, 314: 280, 315: 281, 316: 282, 317: 283, 318: 284, 319: 285, 320: 286, 321: 287, 322: 288, 323: 289, 324: 290, 325: 291, 326: 292, 327: 293, 328: 294, 329: 295, 330: 296, 331: 297, 332: 298, 333: 299, 334: 300, 335: 301, 336: 302, 337: 303, 338: 304, 339: 305, 340: 306, 341: 307, 342: 308, 343: 309, 344: 310, 345: 311, 346: 312, 347: 313, 348: 314, 349: 315, 350: 316, 351: 317, 352: 318, 353: 319, 354: 320, 355: 321, 356: 322, 357: 323, 358: 324, 359: 325, 360: 326, 361: 327, 395: 328, 396: 329, 397: 330, 398: 331, 399: 332, 400: 333, 401: 334, 411: 335, 412: 336, 413: 337, 414: 338, 415: 339, 416: 340, 417: 341, 418: 342, 419: 343, 420: 344, 421: 345, 422: 346, 423: 347, 424: 348, 425: 349, 426: 350, 427: 351, 428: 352, 429: 353, 430: 354, 431: 355, 432: 356, 433: 357, 434: 358, 435: 359, 436: 360, 437: 361, 438: 362, 439: 363, 440: 364, 441: 365, 442: 366, 443: 367, 444: 368, 445: 369, 446: 370, 447: 371, 448: 372, 449: 373, 450: 374, 451: 375, 452: 376, 453: 377, 454: 378, 455: 379, 456: 380, 457: 381, 458: 382, 459: 383, 460: 384, 461: 385, 462: 386, 463: 387, 464: 388, 465: 389, 466: 390, 467: 391, 468: 392, 469: 393, 470: 394, 471: 395, 472: 396, 473: 397, 474: 398, 475: 399, 476: 400, 477: 401, 478: 402, 479: 403, 480: 404, 481: 405, 482: 406, 483: 407, 484: 408, 485: 409, 486: 410, 487: 411, 488: 412, 489: 413, 490: 414, 491: 415, 492: 416, 493: 417, 494: 418, 495: 419, 496: 420, 497: 421, 498: 422, 499: 423, 500: 424, 501: 425, 502: 426, 503: 427, 504: 428, 505: 429, 506: 430, 507: 431, 508: 432, 509: 433, 510: 434, 511: 435, 512: 436, 513: 437, 514: 438, 515: 439, 516: 440, 517: 441, 518: 442, 519: 443, 520: 444, 521: 445, 522: 446, 523: 447, 524: 448, 525: 449, 526: 450, 527: 451, 528: 452, 529: 453, 530: 454, 531: 455, 532: 456, 533: 457, 534: 458, 535: 459, 536: 460, 537: 461, 538: 462, 539: 463, 540: 464, 541: 465, 542: 466, 543: 467, 544: 468, 545: 469, 546: 470, 547: 471, 548: 472, 549: 473, 550: 474, 551: 475, 552: 476, 553: 477, 554: 478, 555: 479, 556: 480, 557: 481, 558: 482, 559: 483, 560: 484, 561: 485, 562: 486, 563: 487, 564: 488, 565: 489, 566: 490, 567: 491, 568: 492, 569: 493, 570: 494, 581: 495, 582: 496, 583: 497, 584: 498, 585: 499, 586: 500, 587: 501, 588: 502, 589: 503, 590: 504, 591: 505, 592: 506, 593: 507, 594: 508, 595: 509, 596: 510, 597: 511, 598: 512, 599: 513, 600: 514, 601: 515, 602: 516, 603: 517, 604: 518, 605: 519, 606: 520, 607: 521, 608: 522, 609: 523, 610: 524, 611: 525, 612: 526, 613: 527, 614: 528, 615: 529, 616: 530, 617: 531, 618: 532, 619: 533, 620: 534, 621: 535, 622: 536, 623: 537, 624: 538, 625: 539, 626: 540, 627: 541, 628: 542, 629: 543, 630: 544, 631: 545, 632: 546, 633: 547, 634: 548, 635: 549, 636: 550, 637: 551, 638: 552, 639: 553, 640: 554, 641: 555, 642: 556, 643: 557, 644: 558, 645: 559, 646: 560, 647: 561, 648: 562, 649: 563, 650: 564, 651: 565, 652: 566, 653: 567, 654: 568, 655: 569, 656: 570, 657: 571, 658: 572, 659: 573, 660: 574, 661: 575, 662: 576, 663: 577, 664: 578, 665: 579, 666: 580, 667: 581, 668: 582, 669: 583, 670: 584, 671: 585, 672: 586, 673: 587, 674: 588, 675: 589, 676: 590, 677: 591, 678: 592, 679: 593, 680: 594, 681: 595, 682: 596, 683: 597, 684: 598, 685: 599, 686: 600}
362 >= 35
pdb_end_cre = mapping.get(362, np.nan)
print(pdb_end_cre)
if np.isnan(pdb_end_cre):
    print("va bien")

nan
va bien


### random checks after some corrections

In [13]:
merged_df.sample(5)

,msa,uniprot,pdb,term_id_cre,start_cre,end_cre,term_id_kd,start_kd,end_kd,chain,mapping,pdb_start_cre,pdb_end_cre,pdb_start_kd,pdb_end_kd
702,P49137_60,P49137,3gok,cre90,339,353,kd89,64,325,H,"{46: 1, 47: 2, 48: 3, 49: 4, 50: 5, 51: 6, 52:...",279,290,19,265
1029,Q15418_60,Q15418,4nif,cre138,687,706,kd138,418,675,A,"{413: 1, 414: 2, 415: 3, 416: 4, 417: 5, 418: ...",272,291,6,260
337,P00533_60,P00533,7jxp,cre39,965,998,kd39,712,979,F,"{700: 1, 701: 2, 702: 3, 703: 4, 704: 5, 705: ...",252,285,13,266
1031,Q15418_60,Q15418,2wnt,cre138,687,706,kd138,418,675,A,"{416: 1, 417: 2, 418: 3, 419: 4, 420: 5, 421: ...",260,279,3,248
485,P28482_60,P28482,7w5o,cre66,173,196,kd66,25,313,A,"{11: 1, 12: 2, 13: 3, 14: 4, 15: 5, 16: 6, 17:...",163,186,15,303


In [9]:
merged_df[(merged_df.pdb == '3gok') & (merged_df.chain == 'H')].mapping[702]

'{46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121, 122: 122, 123: 123, 124: 124, 125: 125, 126: 126, 127: 127, 128: 128, 129: 129, 130: 130, 131: 131, 132: 132, 133: 133, 134: 134, 135: 135, 136: 136, 137: 137, 138: 138, 139: 139, 140: 140, 141: 141, 142: 142, 143: 143, 144: 144, 145: 145, 146: 146, 147: 147, 148: 148, 149: 149, 150: 150, 151: 151, 152: 152, 153: 153, 154: 154, 155: 155, 156: 1

In [17]:
merged_df[(merged_df.pdb == '3gok') & (merged_df.chain == 'H')].mapping[702]

'{46: 1, 47: 2, 48: 3, 49: 4, 50: 5, 51: 6, 52: 7, 53: 8, 54: 9, 55: 10, 56: 11, 57: 12, 58: 13, 59: 14, 60: 15, 61: 16, 62: 17, 63: 18, 64: 19, 65: 20, 66: 21, 67: 22, 68: 23, 69: 24, 70: 25, 71: 26, 72: 27, 73: 28, 74: 29, 75: 30, 76: 31, 77: 32, 78: 33, 79: 34, 80: 35, 81: 36, 82: 37, 83: 38, 84: 39, 85: 40, 86: 41, 87: 42, 88: 43, 89: 44, 90: 45, 91: 46, 92: 47, 93: 48, 94: 49, 95: 50, 96: 51, 97: 52, 98: 53, 99: 54, 100: 55, 101: 56, 102: 57, 103: 58, 104: 59, 105: 60, 106: 61, 107: 62, 108: 63, 109: 64, 110: 65, 111: 66, 112: 67, 113: 68, 114: 69, 115: 70, 116: 71, 117: 72, 118: 73, 119: 74, 120: 75, 121: 76, 122: 77, 123: 78, 124: 79, 125: 80, 126: 81, 127: 82, 128: 83, 129: 84, 130: 85, 131: 86, 132: 87, 133: 88, 134: 89, 135: 90, 136: 91, 137: 92, 138: 93, 139: 94, 140: 95, 141: 96, 142: 97, 143: 98, 144: 99, 145: 100, 146: 101, 147: 102, 148: 103, 149: 104, 150: 105, 151: 106, 152: 107, 153: 108, 154: 109, 155: 110, 156: 111, 157: 112, 158: 113, 159: 114, 160: 115, 161: 116, 

In [20]:
check_3gok =  mapping_coordinates_uniprot_pdb[(mapping_coordinates_uniprot_pdb.pdb == '3gok') & (mapping_coordinates_uniprot_pdb.chain == 'H')]
check_3gok

,uniprot,pdb,chain,alignment,uniprot_seq,pdb_seq,score,start,end,mapping
68,P49137,3gok,H,Alignment(seqA='MLSNSQGQSPPVPFPAPAPPPQPPTPALPH...,MLSNSQGQSPPVPFPAPAPPPQPPTPALPHPPAQPPPPPPQQFPQF...,---------------------------------------------F...,567.4,0,400,"{46: 1, 47: 2, 48: 3, 49: 4, 50: 5, 51: 6, 52:..."


In [22]:
check_3gok.alignment.values

array(["Alignment(seqA='MLSNSQGQSPPVPFPAPAPPPQPPTPALPHPPAQPPPPPPQQFPQFHVKSGLQIKKNAIIDDYKVTSQVLGLGINGKVLQIFNKRTQEKFALKMLQDCPKARREVELHWRASQCPHIVRIVDVYENLYAGRKCLLIVMECLDGGELFSRIQDRGDQAFTEREASEIMKSIGEAIQYLHSINIAHRDVKPENLLYTSKRPNAILKLTDFGFAKETTSHNSLTTPCYTPYYVAPEVLGPEKYDKSCDMWSLGVIMYILLCGYPPFYSNHGLAISPGMKTRIRMGQYEFPNPEWSEVSEEVKMLIRNLLKTEPTQRMTITEFMNHPWIMQSTKVPQTPLHTSRVLKEDKERWEDVKEEMTSALATMRVDYEQIKIKKIEDASNPLLLKRRKKARALEAAALAH', seqB='---------------------------------------------FHVKSGLQIKKNAIIDDYKVTSQVLGLGINGKVLQIFNKRTQEKFALKMLQDCPKARREVELHWRASQCPHIVRIVDVYENLYAGRKCLLIVMECLDGGELFSRIQDRGDQAFTEREASEIMKSIGEAIQYLHSINIAHRDVKPENLLYTSKRPNAILKLTDFGFAKET-----------TPYYVAPEVLGPEKYDKSCDMWSLGVIMYILLCGYPPFYSN----ISPGMKTRIRMGQYEFPNPEWSEVSEEVKMLIRNLLKTEPTQRMTITEFMNHPWIMQSTKVPQTPLHTSRVLKEDKERWE--------------------------------------------------', score=567.3999999999986, start=0, end=400)"],
      dtype=object)

In [24]:
check_3gok.mapping.values

array(['{46: 1, 47: 2, 48: 3, 49: 4, 50: 5, 51: 6, 52: 7, 53: 8, 54: 9, 55: 10, 56: 11, 57: 12, 58: 13, 59: 14, 60: 15, 61: 16, 62: 17, 63: 18, 64: 19, 65: 20, 66: 21, 67: 22, 68: 23, 69: 24, 70: 25, 71: 26, 72: 27, 73: 28, 74: 29, 75: 30, 76: 31, 77: 32, 78: 33, 79: 34, 80: 35, 81: 36, 82: 37, 83: 38, 84: 39, 85: 40, 86: 41, 87: 42, 88: 43, 89: 44, 90: 45, 91: 46, 92: 47, 93: 48, 94: 49, 95: 50, 96: 51, 97: 52, 98: 53, 99: 54, 100: 55, 101: 56, 102: 57, 103: 58, 104: 59, 105: 60, 106: 61, 107: 62, 108: 63, 109: 64, 110: 65, 111: 66, 112: 67, 113: 68, 114: 69, 115: 70, 116: 71, 117: 72, 118: 73, 119: 74, 120: 75, 121: 76, 122: 77, 123: 78, 124: 79, 125: 80, 126: 81, 127: 82, 128: 83, 129: 84, 130: 85, 131: 86, 132: 87, 133: 88, 134: 89, 135: 90, 136: 91, 137: 92, 138: 93, 139: 94, 140: 95, 141: 96, 142: 97, 143: 98, 144: 99, 145: 100, 146: 101, 147: 102, 148: 103, 149: 104, 150: 105, 151: 106, 152: 107, 153: 108, 154: 109, 155: 110, 156: 111, 157: 112, 158: 113, 159: 114, 160: 115, 161

### Check start and end ranges

In [10]:
merged_df[merged_df.pdb_start_cre > merged_df.pdb_end_cre]

,msa,uniprot,pdb,term_id_cre,start_cre,end_cre,term_id_kd,start_kd,end_kd,chain,mapping,pdb_start_cre,pdb_end_cre,pdb_start_kd,pdb_end_kd
27,O96013_60,O96013,5upk,cre32,1,130,kd34,321,572,B,"{300: 300, 301: 301, 302: 302, 303: 303, 304: ...",300,43,321,572
28,O96013_60,O96013,4l67,cre32,1,130,kd34,321,572,A,"{300: 300, 301: 301, 302: 302, 303: 303, 304: ...",300,43,321,572
30,O96013_60,O96013,4fii,cre32,1,130,kd34,321,572,A,"{299: 299, 300: 300, 301: 301, 302: 302, 303: ...",299,55,321,572
32,O96013_60,O96013,4fif,cre32,1,130,kd34,321,572,A,"{296: 296, 297: 297, 298: 298, 299: 299, 300: ...",296,54,321,572
71,P00519_60,P00519,2hz4,cre36,61,233,kd37,242,493,A,"{234: 234, 235: 235, 236: 236, 237: 237, 238: ...",234,233,242,493
790,P53355_60,P53355,1ig1,cre95,292,301,kd95,13,275,A,"{2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: ...",291,277,13,275
791,Q80YE7_60,P53355,1ig1,cre95,292,301,kd95,13,275,A,"{2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: ...",291,277,13,275
792,P53355_60,P53355,1jkl,cre95,292,301,kd95,13,275,A,"{2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: ...",291,277,13,275
793,Q80YE7_60,P53355,1jkl,cre95,292,301,kd95,13,275,A,"{2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: ...",291,277,13,275
794,P53355_60,P53355,1jks,cre95,292,301,kd95,13,275,A,"{2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: ...",291,277,13,275


In [11]:
merged_df[merged_df.pdb == '5upk']

,msa,uniprot,pdb,term_id_cre,start_cre,end_cre,term_id_kd,start_kd,end_kd,chain,mapping,pdb_start_cre,pdb_end_cre,pdb_start_kd,pdb_end_kd
26,O96013_60,O96013,5upk,cre32,1,130,kd34,321,572,A,"{4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 1...",4,43,375,43
27,O96013_60,O96013,5upk,cre32,1,130,kd34,321,572,B,"{300: 300, 301: 301, 302: 302, 303: 303, 304: ...",300,43,321,572


---

### calcular

In [8]:
pdb_list = merged_df[['msa', 'uniprot', 'pdb', 'chain', 'term_id_cre', 'start_cre', 'end_cre',
       'term_id_kd', 'start_kd', 'end_kd',  'pdb_start_cre',
       'pdb_end_cre', 'pdb_start_kd', 'pdb_end_kd']].pdb.unique().tolist()

In [8]:
msa_regions_pdbs[msa_regions_pdbs.pdb.isin(pdb_list)].msa.nunique() # 58 MSAs

58

In [9]:
to_calculate =  merged_df[['msa', 'uniprot', 'pdb', 'chain', 'term_id_cre', 'start_cre', 'end_cre',
       'term_id_kd', 'start_kd', 'end_kd',  'pdb_start_cre',
       'pdb_end_cre', 'pdb_start_kd', 'pdb_end_kd', 'mapping']].dropna() #subset=['pdb_start_cre', 'pdb_end_cre']
to_calculate

,msa,uniprot,pdb,chain,term_id_cre,start_cre,end_cre,term_id_kd,start_kd,end_kd,pdb_start_cre,pdb_end_cre,pdb_start_kd,pdb_end_kd,mapping
0,A5K0N4_60,A5K0N4,5f0a,A,cre10,1,22,kd10,534,791,1,22,534,791,"{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 8: 8, 9: 9, 10:..."
1,Q8I719_60,A5K0N4,5f0a,A,cre10,1,22,kd10,534,791,1,22,534,791,"{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 8: 8, 9: 9, 10:..."
2,A5K0N4_60,A5K0N4,5dzc,A,cre10,1,22,kd10,534,791,1,22,534,791,"{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 8: 8, 9: 9, 10:..."
3,Q8I719_60,A5K0N4,5dzc,A,cre10,1,22,kd10,534,791,1,22,534,791,"{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 8: 8, 9: 9, 10:..."
4,A5K0N4_60,A5K0N4,5dyl,A,cre10,1,22,kd10,534,791,1,22,534,791,"{1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 8: 8, 9: 9, 10:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,Q9UIK4_60,Q9UIK4,1z9x,A,cre273,292,301,kd273,23,285,282,291,13,275,"{12: 2, 13: 3, 14: 4, 15: 5, 16: 6, 17: 7, 18:..."
1074,Q8VDF3_60,Q9UIK4,6paw,A,cre273,292,301,kd273,23,285,282,291,13,275,"{14: 4, 15: 5, 16: 6, 17: 7, 18: 8, 19: 9, 20:..."
1075,Q9UIK4_60,Q9UIK4,6paw,A,cre273,292,301,kd273,23,285,282,291,13,275,"{14: 4, 15: 5, 16: 6, 17: 7, 18: 8, 19: 9, 20:..."
1076,Q91VB2_60,Q96NX5,2jam,A,cre243,277,317,kd243,23,277,278,303,23,278,"{18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 2..."


In [10]:
print(f"There are {to_calculate.pdb.nunique()} PDB structures, corresponding to {to_calculate.uniprot.nunique()} proteins in {to_calculate.msa.nunique()} MSAs")

There are 828 PDB structures, corresponding to 49 proteins in 58 MSAs


In [18]:
to_calculate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1078 entries, 0 to 1077
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   msa            1078 non-null   object
 1   uniprot        1078 non-null   object
 2   pdb            1078 non-null   object
 3   chain          1078 non-null   object
 4   term_id_cre    1078 non-null   object
 5   start_cre      1078 non-null   int64 
 6   end_cre        1078 non-null   int64 
 7   term_id_kd     1078 non-null   object
 8   start_kd       1078 non-null   int64 
 9   end_kd         1078 non-null   int64 
 10  pdb_start_cre  1078 non-null   int64 
 11  pdb_end_cre    1078 non-null   int64 
 12  pdb_start_kd   1078 non-null   int64 
 13  pdb_end_kd     1078 non-null   int64 
dtypes: int64(8), object(6)
memory usage: 126.3+ KB


In [10]:
to_calculate.pdb_start_cre = to_calculate.pdb_start_cre.apply(int)
to_calculate.pdb_end_cre = to_calculate.pdb_end_cre.apply(int)
to_calculate.pdb_start_kd = to_calculate.pdb_start_kd.apply(int)
to_calculate.pdb_end_kd = to_calculate.pdb_end_kd.apply(int)

In [22]:
#to_calculate.to_csv('../datasets/to_calculate_distances.tsv',sep= '\t', index= False)

---

In [26]:
to_calculate[to_calculate.pdb == '1a06']

,msa,uniprot,pdb,chain,term_id_cre,start_cre,end_cre,term_id_kd,start_kd,end_kd,pdb_start_cre,pdb_end_cre,pdb_start_kd,pdb_end_kd
1042,Q63450_60,Q63450,1a06,A,cre183,285,312,kd183,20,276,285,312,20,276
1043,Q91YS8_60,Q63450,1a06,A,cre183,285,312,kd183,20,276,285,312,20,276


In [31]:
to_calculate.iloc[:3,:]

,msa,uniprot,pdb,chain,term_id_cre,start_cre,end_cre,term_id_kd,start_kd,end_kd,pdb_start_cre,pdb_end_cre,pdb_start_kd,pdb_end_kd
0,A5K0N4_60,A5K0N4,5f0a,A,cre10,1,22,kd10,534,791,1,22,534,791
1,Q8I719_60,A5K0N4,5f0a,A,cre10,1,22,kd10,534,791,1,22,534,791
2,A5K0N4_60,A5K0N4,5dzc,A,cre10,1,22,kd10,534,791,1,22,534,791


In [13]:
hello()

'alv'

In [27]:
distances_all_atoms(path= '../datasets/pdb_files/uncompressed/', output= '../datasets/interatomic_distances_both_regions_kd_cre/', pdb_coord= to_calculate) # .iloc[:3,:]

distances for 1a06 calculated
writing distances as 1a06.csv
distances for 1f3m calculated
writing distances as 1f3m.csv
distances for 1gzk calculated
writing distances as 1gzk.csv
distances for 1gzn calculated
writing distances as 1gzn.csv
distances for 1gzo calculated
writing distances as 1gzo.csv
distances for 1ig1 calculated
writing distances as 1ig1.csv
distances for 1jkl calculated
writing distances as 1jkl.csv
distances for 1jks calculated
writing distances as 1jks.csv
distances for 1m14 calculated
writing distances as 1m14.csv
distances for 1m17 calculated
writing distances as 1m17.csv
distances for 1mrv calculated
writing distances as 1mrv.csv
distances for 1mry calculated
writing distances as 1mry.csv
distances for 1nxk calculated
writing distances as 1nxk.csv
distances for 1ny3 calculated
writing distances as 1ny3.csv
'C'
'C'
distances for 1opl calculated
writing distances as 1opl.csv
distances for 1p4f calculated
writing distances as 1p4f.csv
distances for 1pkg calculated
wr

In [14]:
msa_regions_pdbs.msa.value_counts()

P00533_60    215
P28482_60    123
P00523_60     70
Q02750_60     60
P35968_60     42
P00519_60     39
Q14680_60     31
P31749_60     28
P49137_60     26
P49138_60     26
Q13131_60     22
P54645_60     22
P10721_60     22
P07333_60     19
P31751_60     16
Q80YE7_60     15
P53355_60     15
Q8VDF3_60      9
Q09137_60      8
Q8BRK8_60      8
P54646_60      8
O75582_60      7
A5K0N4_60      7
Q8I719_60      7
P36888_60      6
Q9UIK4_60      6
O96013_60      5
P43403_60      5
Q8IU85_60      5
Q00944_60      5
Q8BW96_60      5
Q91YS8_60      5
Q13188_60      5
Q16644_60      5
P51812_60      5
Q13557_60      5
Q14012_60      4
P45985_60      3
Q13043_60      3
Q15418_60      3
Q01974_60      2
Q8IBS5_60      2
Q61846_60      2
Q6PHZ2_60      1
Q16566_60      1
Q91VB2_60      1
Q08E52_60      1
Q13153_60      1
Q13976_60      1
Q13555_60      1
P29323_60      1
O74536_60      1
P04049_60      1
Q96NX5_60      1
Q13554_60      1
Q63450_60      1
P36507_60      1
O77676_60      1
Name: msa, dty